<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/ACLU_Project_Part2Transform_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

#GoogleDriveBase = "/content/drive/My Drive/" #Kyle
GoogleDriveBase = "/content/drive/MyDrive/content/drive/My Drive/Projects/" #Anupriya
#GoogleDriveBase = "/content/drive/MyDrive/" #Mackenzie
#WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Kyle
WorkingDirectory = GoogleDriveBase + 'ACLU/' #Anupriya
#WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Mackenzie
WorkingFiles = WorkingDirectory + 'WorkingFiles/'
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"


#Make the necessary folders for the script to run.
ListOfAllRequiredDirectories = [WorkingDirectory + 'Pickle', 
                          WorkingDirectory + 'WorkingFiles',
                          WorkingDirectory + 'AdditionalData',
                          WorkingDirectory + 'ACLUData']

for folder in ListOfAllRequiredDirectories:
  RunningPath = GoogleDriveBase + folder + "/"
  Path(RunningPath).mkdir(parents=True, exist_ok=True)


In [ ]:
import numpy
import sys
import csv
import string
import pandas as pd
import gc
import re
import string

In [ ]:
#dfVoters = pd.read_csv(WorkingFiles + "dfVoters.csv", dtype=str) #Not needed for the transformations
dfMerged = pd.read_csv(WorkingFiles + "dfMerged.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17,18,19,20,21,23,27,29,31,32,34,36,39,40,42,44,46,48,49,50,52,53,54,55,56,57,58,60,61,65,66,68,70,71,72,73,74,75,76,77,78,79,80,81) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


This is an example of duplicate data we found by combining the data files. It apepars that if a person is included in multiple outreach attempts, their full data will be included in each outreach attempt email. 

We went back and verified that this was expected behavior with the UCLA. Just make a note of it for now, we will be actually removing it further into this notebook.

In [ ]:
dfMerged[dfMerged["Voter File VANID"] == 11]

,Unnamed: 0,index,Voter File VANID,StateFileID,General20,General19,General18,General17,General16,General15,General14,General12,General11,General10,General09,General08,General06,General04,General02,General00,General98,Municipal13,PresPrimary20,PresPrimary20Party,PresPrimary16,PresPrimary16Party,PresPrimary12,PresPrimary12Party,PresPrimary08,PresPrimary08Party,Primary20,Primary20Party,Primary19Party,Primary18,Primary18Party,Primary17,Primary17Party,Primary16,Primary16Party,Primary15,...,Primary12Party,Primary11,Primary11Party,Primary10,Primary10Party,Primary09,Primary09Party,Primary08,Primary08Party,Primary06,Primary04,Primary02,Primary01,Primary00,Primary98,Special20,Special19,Special18,Special17,Special16,Special15,Special13,Special12,Special11,Special10,Special09,Special08,Special07,Special06,Special05,Special04,Special03,Special02,Special01,Special00,Special99,Special98,Ballot_Application,Voting_Nov_Gen,VotedStatus
1115312,1115312,76968,11,106369799.0,A,NaN,P,NaN,P,NaN,NaN,NaN,NaN,NaN,NaN,P,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1172222,1172222,42602,11,106369799.0,A,NaN,P,NaN,P,NaN,NaN,NaN,NaN,NaN,NaN,P,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A: Absentee

P: Polls (if there is a D or R, that is specifying which party they voted during the primary)

M: This one is new to me. Let me check in with our database administrator

Y: Yes—indicates that they voted, but we don’t have data on how they voted (in person vs. absentee)

In [ ]:
primary_key = ["Voter File VANID"]
columns_processed = []

def ReshapeDataframeYears(df):
  ColumnsToTransform = ["Special##", "General##", "Primary##", "PresPrimary##", "PresPrimary##Party", "Primary##Party", "Municipal##"]
  #ColumnsToTransform = ["Special##"]
  melted_dataframes = []
  for name in ColumnsToTransform:
    ColumnsForName = []
    years = []
    number_index = name.index("##")
    for column in df.columns:
      if len(column) < len(name):
        continue
      first = column[0:number_index]
      name_first = name[0:number_index]
      number = column[number_index:number_index+2]
      if len(column) - number_index+2 == 0:
        second = ""
        name_second = ""
      else:
        second = column[number_index+2:len(column)]
        name_second = name[number_index+2:len(name)]

      if first == name_first and second == name_second:
        ColumnsForName.append(column)
        columns_processed.append(column)
        year = column[len(first):len(first)+2]

        if int(year) < 50:
          year = "20" + year
        else:
          year = "19" + year
        years.append(year)
        #print(int(year))
    df_reduced = df[primary_key + ColumnsForName]
    df_reduced.columns = primary_key + years
    df_melted = pd.melt(df_reduced, id_vars=primary_key, value_vars=years)
    
    #Restructure the columns
    final_columns = primary_key.copy()
    final_columns.append("year")
    final_columns.append("participation")
    df_melted.columns = final_columns
    df_melted["electiontype"] = name
    #Drop records where we don't have participation data. 
    df_melted = df_melted[df_melted['participation'].notna()] 
    #print(df_melted.head())
    melted_dataframes.append(df_melted)
  return melted_dataframes

def CombineMeltedDataframes(melted_dataframes):
  #Now join them all back together
  #Rather silly that this is my method, but I tried a few different versions to find the most memory efficient way to do this
  return pd.concat(melted_dataframes)

melteddfs = ReshapeDataframeYears(dfMerged)
ColumnsThatWereNotPartOfTheRestructuredColumns = set(dfMerged.columns) - set(columns_processed)


The following cell is just here fora  visual check to ensure that we aren't going to be leaving beyind any columns that we should care about. 

In [ ]:
for column in ColumnsThatWereNotPartOfTheRestructuredColumns:
  print(column)
  print(dfMerged[column].unique())

dfMerged[ColumnsThatWereNotPartOfTheRestructuredColumns].head()

Unnamed: 0
[      0       1       2 ... 3196828 3196829 3196830]
index
[     0      1      2 ... 343455 343456 343457]
Voter File VANID
[   82502    85510    71010 ... 19713544 19717863 20854760]
Voting_Nov_Gen 
[nan 'Yes' 'Maybe/Unsure' 'No']
Ballot_Application
[nan 'No - need help/info' 'Yes' 'No - voting at polls' 'Not Asked']
VotedStatus
[nan]
StateFileID
[3.07112000e+05 3.12808000e+05 2.84376000e+05 ... 3.46065100e+07
 3.47575320e+07 1.59840494e+08]


,Unnamed: 0,index,Voter File VANID,Voting_Nov_Gen,Ballot_Application,VotedStatus,StateFileID
0,0,0,82502,NaN,NaN,NaN,307112.0
1,1,1,85510,NaN,NaN,NaN,312808.0
2,2,2,71010,NaN,NaN,NaN,284376.0
3,3,3,44565,NaN,NaN,NaN,229902.0
4,4,4,128077,NaN,NaN,NaN,391301.0


In [ ]:
del dfMerged
gc.collect()

dfreshaped = CombineMeltedDataframes(melteddfs)

del melteddfs
gc.collect()

0

In [ ]:
dfreshaped.head()

,Voter File VANID,year,participation,electiontype
0,82502,2020,P,Special##
2,71010,2020,A,Special##
3,44565,2020,A,Special##
4,128077,2020,A,Special##
6,83085,2020,A,Special##


In [ ]:
dfreshapedUnique = dfreshaped.drop_duplicates()

In [ ]:
dfreshapedUnique = dfreshapedUnique.sort_values(["Voter File VANID", "year"])

In [ ]:
dfreshapedUnique.to_csv(WorkingFiles + "dfreshapedUnique.csv", encoding='utf-8')

In [ ]:
import altair as alt

df_summary = dfreshaped.groupby(["year", "electiontype"]).count().reset_index()
alt.Chart(df_summary).mark_line().encode(
    x='year',
    y='participation',
    color='electiontype',
)


#df_summary = df_reshaped.groupby(["year"]).count().reset_index()
#ax = plt.gca()
#df_summary.plot(kind='line',x='year',y='Special##',ax=ax)
#df_summary.head()

alt.Chart(...)

In [ ]:
check = pd.read_csv(WorkingFiles + "dfreshapedUnique.csv", encoding='utf-8')
check.head()

,Unnamed: 0,Voter File VANID,year,participation,electiontype
0,39477284,11,2006,Y,General##
1,36280453,11,2008,P,General##
2,13902636,11,2016,P,General##
3,7508974,11,2018,P,General##
4,1115312,11,2020,A,Special##
